# Chaînes de Markov

Nous étudierons des chaînes de Markov. Une chaîne de Markov est un modèle probabiliste relativement simple mais très puissant. Nous allons créer un modèle simple pour étudier le temps, et pour synthétiser le langage naturel. 

Un chaîne de Markov est une suite $X_0,X_1,X_2,\dots$ de variables aléatoires à valeurs dans un espace d'états discrets $E=\{0,1,\dots,k\}$. On peut l'interpréter commme un processus dont l'état  est $X_i$ au temps $t=i$. Nous étudions la probabilité
$$ P(X_n = i \mid X_0=i_0,X_1=i_1,\dots,X_{n-1}=i_{n-1}). $$
Si les variables sont indépendantes cette probabilité est égale à $P(X_n=i)$ mais ce n'est pas très intéressant. Au lieu de cela, nous supposons que 
$$ P(X_n = i \mid X_0=i_0,X_1=i_1,\dots,X_{n-1}=i_{n-1}) = P(X_n = i \mid X_{n-1}=i_{n-1}), $$
c'est-à-dire, si on veut prédire $X_n$ sur la base de l'histoire des états, il suffit seulement de connaître l'état précédent.

# Exercice 1
### Modèles météorologiques et distributions stationnaires

On considère un exemple. Supposons que le temps du jour suivant dépend seulement du temps de la veille. Supposons aussi qu'il y a seulement trois types de temps: ensoleillé, couvert ou pluvieux (E, C, P). On a les informations suivantes:
- Si aujourd'hui est ensoleillé, demain c'est aussi ensoleillé avec p=0.7, couvert avec p=0.2, et pluvieux avec p=0.1
- Si aujourd'hui est couvert, demain c'est aussi couvert avec p=0.5, ensoleillé avec p=0.25, et pluvieux avec p=0.25
- Si aujourd'hui est pluvieux, demain c'est aussi pluvieux avec p=0.4, ensoleillé avec p=0.4, et couvert avec p=0.2

On met ces informations dans une matrice:
$$
    P = \begin{pmatrix}
        0.7 & 0.2 & 0.1\\
        0.25 & 0.5 & 0.25\\
        0.2 & 0.4 & 0.4
    \end{pmatrix}
$$
Dénotons la temps du jour $n$ par $X_n$. Les variables aléatoires $\{X_n\}_{n\in\mathbb N}$ forment une chaîne de Markov. La matrice $P$ représente les probabilités de transition. C'est-à-dire, $P_{ij} = P(X_n=j\mid X_{n-1}=i)$, la probabilité de transiter de l'état $i$ à l'état $j$. Notons que la somme des lignes de cette matrice sont toutes égales à 1. C'est parce que $\sum_jP_{ij} = \sum_j P(X_n=j\mid X_{n-1}=i) = 1$.


In [1]:
import numpy as np # on va utiliser numpy pour des calculs numériques

# La matrice de transitions
Pij = np.array([[0.7,0.2,0.1],[0.25,0.5,0.25],[0.2,0.4,0.4]])

# On assigne un numéro de chaque état
states_dictionary = {'E':0, 'C':1 ,'P':2}
states = ['E','C','P']

def simulate_weather(n=100, initial_state = 'E'):
    """Donne un simulation de temps pour `n` jours avec état initial `initial_state`"""
    
    current_state = states_dictionary[initial_state] # état actuel 
    output_states = [initial_state] # liste de tous les états
    for i in range(n):
        transition_probabilities = Pij[current_state] # obtenir probabilités de transition
        
        # échantillonner la distribution multinomiale
        current_state = np.random.choice(range(3),p=transition_probabilities)
        
        # ajoute le nouvel état à la liste
        output_states.append(states[current_state])
    return output_states

In [2]:
print(simulate_weather(n=20,initial_state='E'))

['E', 'E', 'E', 'E', 'E', 'E', 'E', 'P', 'C', 'C', 'C', 'P', 'C', 'P', 'C', 'P', 'P', 'E', 'E', 'E', 'E']


## Exercice 1 a)
> Écrire une fonction `fraction_count` qui calcule la fraction d'occurence de tous les élements d'une liste. La fonction doit renvoyer le résultat comme un dictionnaire.

Exemple: `fraction_count(['A', 'B', 'A', 'C'])` doit renvoyer `{'A':0.5, 'B':0.25, 'C':0.25}`.

In [3]:
def fraction_count(input_list):
    """Renvoie la fraction de tous les élements de input_list comme un dictionnaire"""
    fractions = dict()
    total_length = len(input_list)
    
    
#on regarde chaque élément dans l'input_list
    for element in input_list :
        #on crée un compteur qui va compter le nombre de répétitions de l'élément dans la liste input_liste
        compteur = 0
        #pour le chaque élément de input_liste on va parcourir tous les éléments de la input_liste
        for elementCopie in input_list :
            #on incrémente le compteur à chaque fois que l'élément apparaît dans la liste
            if element == elementCopie :
                compteur +=1
        #on calcule le nombre d'apparitions de l'élément par rapport au nombre d'éléments total dans la liste
        nombreApparition = compteur/total_length
        #on ajoute l'élément et son nombre d'apparition dans le dictionnaire
        fractions[element] = nombreApparition

    
    return fractions

In [4]:
# Tests vérifiants si votre code est correct
l = [1,2,4,1,2,'A','B',1]
if fraction_count(l)[1]!=0.375:
    print('Le code n\'est pas correct')
elif sum(fraction_count(l).values())!=1.0:
    print('Le code n\'est pas correct. La somme des fractions doit être égal a 1')
else:
    print('Le code passe tous les tests !')

Le code passe tous les tests !


Avec cette fonction, on peut afficher la distribution comme un diagramme circulaire.

In [5]:
import matplotlib.pyplot as plt

fractions = fraction_count(simulate_weather(n=10000))
print(fractions)

plt.pie(fractions.values(),labels=fractions.keys(),autopct='%1.1f%%');

{'E': 0.44845515448455153, 'C': 0.34906509349065096, 'P': 0.2024797520247975}


Nous avons obtenu la distribution  $v=(v_0,v_1,v_2)$  de tous les types de temps. Cette distribution est appelée la distribution stationnaire de la chaîne $\{X_n\}$. Ce distribution satisfait $P^\top v=v$, où $P^\top$ est la transposée de la matrice de transition. On va vérifier cette propriété:

In [6]:
# réordonner la liste
fractions_ordered = np.array([fractions['E'], fractions['C'], fractions['P']])

differences = np.transpose(Pij)@fractions_ordered-fractions_ordered

norm = np.linalg.norm(differences)
if norm<0.05:
    print('Succès ; Pv = v')
else:
    print('Pv != v, vérifier le code. ')

Succès ; Pv = v


## Exercice 1 b)

C'est possible d'obtenir la distribution stationnaire directement à partir de la matrice $P$. Notez que $P^\top v=v$ est équivalent à trouver un vecteur propre de $P$ avec valeur propre égale à 1. 
> En utilisant `numpy.linalg.eig`, trouvez la distribution stationnaire. Notez que `numpy.linalg.eig` renvoie des vecteurs propres avec longueur 1, mais pour trouver une distribution nous avons besoin de vecteurs avec somme égale à 1. On a donc besoin de renormaliser. N'oubliez pas de transposer la matrice `Pij` en utilisant `np.transpose`. 

In [11]:
# Écrivez votre code dessus
# --------------------------------------------
import numpy as np
#on stock les valeurs propre de la transposé de P dans "valpropre" et les vecteurs propres correspondants dans "v"
valpropre, v = np.linalg.eig(np.transpose(Pij))
#on affiche les valeur propre pour voir laquelle est = 1
print("les valeurs propres sont : ",valpropre)
v = v/sum(v)
#on remarque en print(valpropre) que la valeur propre en position[0] est = 1, et elle correspond au vecteur propre v[:,0] donc le premier vecteur propre de v
v = v[:,0]
# Tests vérifiants si la distribution stationnaire est correcte
differences = np.transpose(Pij)@v-v
norm = np.linalg.norm(differences)
if norm>1e-6:
    print('La vecteur n\'est pas un vecteur propre avec valeur propre 1.')
elif abs(sum(v)-1)>1e-6:
    print('La somme du vecteur n\'est pas égale a 1')
else:
    print('Succès !')

les valeurs propres sont :  [1.         0.47320508 0.12679492]
Succès !


## Exercice 1 c)

On peut aussi obtenir la distribution stationnaire en calculant la puissance $(P^n)$. Notez que si $P^\top v=v$, on a aussi $(P^n)^\top v =v$ pour tout $n$. Ci-dessous on a calculé $P^{10}$.

In [ ]:
# Calculer la puissance d'une matrice
np.linalg.matrix_power(Pij,10)

> Notez que toutes les rangées de $P^{10}$ sont proches de la distribution stationnaire. 
 Expliquez ce phénomène ci dessus en texte. Encapsulez du texte entre deux signes dollar `$ $` pour écrire des maths.

__VOTRE RESPONSE:__
_______

> <span style="color:red">On sait que si la distribution limmite existe elle est stationnaires, et que cette limmite est egale une matice de transition dont toutes les lignes sont égales entre elles. On a donc que une ligne de cette marice est la distribution stationnaire. C'est pour ça que dans la matice ci-dessus (avec n assez grand) les lignes tendent vers la distribution stationnaire.</span>

_____

# Exercice 2
### Modèles de langage naturel avec des chaînes de Markov

On peut simuler du langage naturel avec des chaînes de Markov. Le texte n'est qu'une suite de lettres $s_0s_1\dots s_n$, avec $s_i$ des lettres comme a,b,c, et de la ponctuation. Une modèle naïf de texte est donc un chaîne de Markov avec $X_n$ le $n$ième lettre de texte. La propriété de Markov
$$P(X_n = i \mid X_0=i_0,X_1=i_1,\dots,X_{n-1}=i_n) = P(X_n = i \mid X_{n-1}=i_{n-1})$$
n'est pas réaliste, mais on peut voir ce qui se passe quand on suppose cela. 

Pour estimer les probabilités de transition $P_{ij}=P(X_n = i \mid X_{n-1}=i_{n-1})$ on va utiliser du texte. La probabilité $P_{ij}$ est égale à la fraction de fois que la lettre $i$ suit la lettre $j$ dans notre texte. C'est-à-dire
$$
P_{ij} = \frac{\#\{j\mid \text{$j$ apparaît dans le texte, et la lettre précédente est $i$}\}}{
\#\{i\mid \text{$i$ apparaît} \}}
$$

On va utiliser les poésies complètes d'Arthur Rimbaud.

Le texte est disponible en ligne:
http://www.gutenberg.org/ebooks/29302

D'abord on va analyser un petit peu le fichier de texte.

In [ ]:
# Prétraitement de texte. Nous voulons seulement les lignes de texte avec de la poésie. 
with open('rimbaud.txt',encoding='utf8') as file_read:
    with open('rimbaud_edited.txt','w',encoding='utf8') as file_write:
        for line in file_read:
            if line[:4]=='    ':  # on voit que seulement ces lignes sont de la poésie
                line_edited = line.lower()  # seulement lettres minuscules 
                line_edited= ' '.join(line_edited.split())  # supprimer les espaces non-utilisés
                line_edited = ''.join([c for c in line_edited if c not in ['_','-']])
                
                if line_edited == 'préface': # tout le texte suivant `préface' n'est pas intéressant
                    break
                if line_edited[:4]!='. . ': # quelques lignes composées seulement de points
                    file_write.write(line_edited+'\n')  # écrire des lignes bonnes dans un fichier

On regarde quelles lettres il y a dans le fichier, et on va donner à chaque lettre un numéro unique. 

In [ ]:
# Donner à chaque lettre un numéro unique dans un dictionnaire
character_dic = dict()

# Compter le nombre de lettres différentes 
character_count = 0

with open('rimbaud_edited.txt',encoding='utf8') as f:
    for line in f:
        for c in line:
            if c not in character_dic: # si la lettre n'es pas encore connue
                character_dic[c] = character_count # donne à la lettre son numéro
                character_count+=1
                
# faire une liste de toutes les lettres
characters = list(character_dic.keys())
                
print('Il y a %d lettres différentes dans la texte:\n' % character_count)
print(character_dic)

Il y a 52 lettres différentes. La matrice de transition va donc être $52\times 52$. Nous pouvons réduire un petit peu les dimensions en supprimant les lettres pas importantes comme `[`et `]`, mais ce n'est pas très important.

## Exercice 2 a) 
> Écrivez du code qui compte toutes les fois où la lettre $i$ suit la lettre $j$ dans le texte pour tous $i$, $j$. On va mettre les taux dans un matrice `Pij_lang`. 

In [ ]:
Pij_lang = np.zeros((character_count, character_count),dtype=np.int32)
with open('rimbaud_edited.txt',encoding='utf8') as f:
    buffer = f.read()  # un chaîne de caractères qui contient le fichier entier   
    
    # Écrivez votre code dessus
    #for c in buffer:
    for c in range(0,len(buffer)-1):
        i=character_dic[buffer[c]]
        j=character_dic[buffer[c+1]]
        Pij_lang[i][j]+=1
        
# renormaliser la matrice pour que la somme de toutes les rangées soit 1.
Pij_lang=Pij_lang/Pij_lang.sum(axis=1)[:,None]

# quelques tests pour confirmer que le code est correct
assert abs(np.mean(Pij_lang)-0.019230769230769232)<0.01
assert abs(np.trace(Pij_lang)-0.8163083883466542)<0.01
assert abs(np.std(Pij_lang)-np.std(Pij_lang))<0.01

Avec la matrice de transition on peut produire du texte synthétique pour simuler la chaîne de Markov comme avant. 

In [ ]:
def generate_text(n=100, initial_state = '\n'):
    current_state = character_dic[initial_state]
    output_states = []
    for i in range(n):
        transition_probabilities = Pij_lang[current_state]
        current_state = np.random.choice(range(character_count),p=transition_probabilities)
        output_states.append(characters[current_state])
    return ''.join(output_states)
print(generate_text(200))

Le texte généré n'est pas encore très bon. La majorité des mots produits n'existent pas. 

On peut améliorer la modèle en relaxant la propriété de Markov. On suppose que la $n$ième lettre dépend des $k$ lettres précédentes, et pas seulement de la seule lettre avant. C'est-à-dire, on suppose que
$$P(X_n = i \mid X_0=i_0,X_1=i_1,\dots,X_{n-1}=i_n) = P(X_n = i \mid X_{n-k}=i_{n-k},\dots, X_{n-1}=i_{n-1}).$$
En pratique $k=3$ ou $k=4$. Quand $k>1$ il y a un bonne définition de matrice de transition, mais elle n'est plus carrée. Supposons que $k=3$, les autres cas sont similaires. On peut définir 
$$P_{(i_1,i_2,i_3),j} = P(X_n = j \mid X_{n-3}=i_{1},X_{n-2}=i_{2},X_{n-1}=i_{3}).$$
Ici $(i_1,i_2,i_3)$ est une suite de trois lettres. Car il y a $52^3 = 14 0608$ séquences comme ça, donc la matrice est $140608\times 52$. C'est un peu grand, donc on a besoin d'utiliser des matrices creuses. Les matrices creuses conservent seulement les entrées de matrice non-nulles, et utilisent beaucoup moins de mémoire.  


## Exercice 2 b)
> Écrivez un code qui compte le nombre total de transitions $(i_1,i_2,i_3,...)\to j$ dans le texte en une matrice creuse. Comptez aussi la fréquence de toutes les lettres individuelles.

In [ ]:
from scipy.sparse import dok_matrix, csr_matrix # pour utiliser des matrices creuses

# Le paramètre k
TIME_HORIZON = 4

def chars_to_index(chars):
    """convertir une suite de lettres (i_1,...,i_k) en indice de matrice"""
    index = 0
    for i,c in enumerate(chars):
        index+=character_dic[c]*(character_count**i)
    return index

# Initialisez la matrice creuse
# La taille est 52^k x 52
Pij_sparse = dok_matrix((character_count**TIME_HORIZON,character_count))

# Comptez aussi la fréquence de toutes les lettres individuelles
character_frequencies = np.zeros(character_count)
with open('rimbaud_edited.txt',encoding='utf8') as f:
    buffer = f.read()
    prev_chars = buffer[:TIME_HORIZON]
    for c in buffer[1:]:
        j = character_dic[c]
        i = chars_to_index(prev_chars)
        
        character_frequencies[j] +=1

        Pij_sparse[i,j] += 1
        
        prev_chars = prev_chars[1:]+c

# renormaliser
character_frequencies=character_frequencies/character_frequencies.sum()

Pij_sparse=Pij_sparse.tocsr() # Convertir en un format de matrice creuse plus vite

Comme avant, avec la matrice de transition on peut produire du texte en simulant la chaîne de Markov. 

In [8]:
def get_probs(chars):
    """renvoyez la rangée de matrice de transition à partir d'un suite de lettres"""
    row = np.array(Pij_sparse[chars_to_index(chars)].todense())
    row = row.reshape(-1)
    row_sum = row.sum()
    if row_sum == 0:  # Si la suite est inconnue, renvoyer les fréquences globales.
        return character_frequencies
    else:
        return row/row_sum

In [9]:
def generate_text(n=100, initial_state = '\n'*TIME_HORIZON):
    current_state = initial_state
    output_states = []
    for i in range(n):
        transition_probabilities = get_probs(current_state)
        new_index = np.random.choice(range(character_count),p=transition_probabilities)
        new_char = characters[new_index]
        output_states.append(new_char)
        current_state = current_state[1:]+new_char
    return ''.join(output_states)

NameError: name 'TIME_HORIZON' is not defined

In [10]:
print(generate_text(n=1000))

NameError: name 'generate_text' is not defined

On voit que le texte généré est mieux. Ce n'est pas encore de la poésie, mais la majorité des mots générés sont orthographiés correctement.